In [1]:
import graphlab

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to jjmmg@yahoo.comand will expire on October 22, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-5476 - Server binary: C:\Users\JJMMG\AppData\Local\Dato\Dato Launcher\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\JJMMG\AppData\Local\Temp\graphlab_server_1445851568.log.0
[INFO] GraphLab Server Version: 1.6.1


In [3]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [4]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [5]:
for e in selected_words:
    products[e] = products['word_count'].apply(lambda d: d[e] if e in d else 0)

In [6]:
sums = {}
for e in selected_words:
    sums[e] = products[e].sum()

In [37]:
print 'Question 1: %s' % (max(sums, key=sums.get))

Question 1: great


In [38]:
print 'Question 2: %s' % (min(sums, key=sums.get))

Question 2: wow


In [9]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [10]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

In [11]:
train_data,test_data = products.random_split(.8, seed=0)

In [12]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 3.904224     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 7.889452     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 9.721557     | 0.92

In [13]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.379021     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 0.651037     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 0.895051     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        |

In [14]:
weights = selected_words_model['coefficients'].sort('value', ascending=False)

In [39]:
print 'Question 3: %s' % (weights[0]['name'])

Question 3: love


In [40]:
print 'Question 4: %s' % (weights[-1]['name'])

Question 4: terrible


In [17]:
sentiment_result = sentiment_model.evaluate(test_data)

In [18]:
selected_words_result = selected_words_model.evaluate(test_data)

In [41]:
print 'Question 5: %s' % (selected_words_result['accuracy'])

Question 5: 0.843141964929


In [42]:
print 'Question 6: %s' % (sentiment_result['accuracy'])

Question 6: 0.916256305549


In [43]:
selected_confusion_matrix = selected_words_result['confusion_matrix']

In [44]:
from __future__ import division

In [45]:
print 'Question 7: %s' % (selected_confusion_matrix[selected_confusion_matrix['target_label'] == 1]['count'].sum() / selected_confusion_matrix['count'].sum())

Question 7: 0.840019216911


In [71]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [72]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [73]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [74]:
print 'Question 9: %s' % (diaper_champ_reviews[0]['predicted_sentiment'])

Question 9: 0.999999937267


In [78]:
print 'Question 10: %s' % (selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability'))

Question 9: [0.796940851290671]


In [82]:
print diaper_champ_reviews[0]['review']

Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend's house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn't any added bulk around the sealing edge of the champ.


In [88]:
d = diaper_champ_reviews[0]['word_count']
for w in sorted(d, key=d.get, reverse=True):
    print w, d[w]

the 17
diaper 7
and 6
a 6
is 4
to 3
really 3
one 3
there 3
baby 3
i 3
absolutly 2
easy 2
best 2
we 2
of 2
into 2
was 2
that 2
champ, 2
in 2
for 2
genie 2
all 1
less 1
friend's 1
(which 1
around 1
deffinite 1
luke 1
champ 1
turns 1
bag 1
quick 1
found 1
where 1
isn't 1
because 1
house 1
are 1
"what 1
what 1
wondered 1
product, 1
pad) 1
3 1
integrated 1
dirty 1
seconds 1
odor 1
little 1
from 1
on 1
chanp 1
turn 1
free, 1
purchase.great 1
reinforced 1
garbage 1
vie 1
economical, 1
smelly 1
changing 1
nursery. 1
use 1
flat. 1
bjorn, 1
needed 1
smell" 1
bulk 1
hesitated 1
graco 1
champ. 1
than 1
loved 1
this 1
work 1
useing 1
can 1
pack 1
purchases 1
bassinet 1
use, 1
at 1
have 1
any 1
if 1
bags 1
comparison, 1
no 1
effective, 1
handed 1
vital 1
tie 1
you 1
smell 1
difficult 1
play 1
added 1
genieplus 1
knew 1
that. 1
eminating 1
ties 1
sealing 1
made. 1
that!! 1
well 1
edge 1
clean 1
time 1
fluerville 1
bag, 1
fabulous.updatei 1


In [95]:
for w in sorted(d, key=d.get, reverse=True):
    if w in selected_words:
        print w, d[w]